In [1]:
server_args = []
import math
import numpy as np
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)
from pydrake.all import (LeafSystem,BasicVector,DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, LinearQuadraticRegulator,
                         Simulator, RigidTransform, CoulombFriction, FindResourceOrThrow, DrakeVisualizer, ConnectContactResultsToDrakeVisualizer,
                         RollPitchYaw, JointIndex, namedview, ConnectMeshcatVisualizer,
                         Value, List, ZeroOrderHold, SpatialAcceleration, RotationMatrix, AbstractValue, ConstantVectorSource)


# if you need to use other class from Drake, you can add them to the list above

In [4]:
builder = DiagramBuilder()
# First add the cart-pole system from a urdf file
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
urdf_path = "./urdfExample_cart_pole.urdf"    
cart_pole = Parser(plant, scene_graph).AddModelFromFile(urdf_path)    
plant.Finalize()

# Add controller (u = 0)
controller = builder.AddSystem(ConstantVectorSource([0]))

# connect to make diagram
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

# set up visualization using meshcat
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, open_browser=True)
diagram = builder.Build()

# start simulation
# UprightState = np.array([0, np.pi, 0, 0])   # the state of the cart-pole is organized as [z, theta, zdot, theta_dot]
# UprightState = np.array([0, np.pi/2, 0, 0])   # the state of the cart-pole is organized as [z, theta, zdot, theta_dot]
UprightState = np.array([0, np.pi/3, 0, 0])   # the state of the cart-pole is organized as [z, theta, zdot, theta_dot]

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()
# context.SetContinuousState(UprightState + np.array([0.1,0.3,0.3,0.1]))  # 有扰动偏移 disturbance
context.SetContinuousState(UprightState)  # 无扰动偏移 No disturbance
simulator.Initialize()

sim_time = 5
meshcat.start_recording() 
simulator.AdvanceTo(sim_time)
meshcat.stop_recording()
meshcat.publish_recording()
simulator.AdvanceTo(sim_time)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6007...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connected to meshcat-server.
